In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statistics import mean


In [12]:
seeds = [42, 121, 78, 52]
nums = [2, 4, 8, 16, 32]
ty = "locution" # "locution" "proposition"


accs =  [[],[],[],[],[],[]]
f1s = [[],[],[],[],[],[]]


In [13]:
for i in range(len(nums)):
    num = nums[i]
    for j in range(len(seeds)):
        seed = seeds[j]
        for run in range(1,4):

            pred_df = pd.read_csv(f"Llama_priming_predictions/predictions/Llama3-8B_{ty}_{num}_{seed}_{run}.tsv", sep="\t")
            preds = [pred.lower().replace("**","").replace(".","").replace("_","").replace("'","").strip() for pred in list(pred_df["prediction"])]
            gold_labels = list(pred_df["gold_label"])

            #prime_df = pd.read_csv(f"primes/{ty}_primes_{num}_{seed}.tsv", sep="\t")
            #prime_labels = list(prime_df["label"])

            #print(set(preds))
            
            for n in range(len(preds)):
                pred = preds[n]
                if "non-contr" in pred: #and pred.lower().replace("'", "").replace(".", "").endswith("non-contradiction"):
                    #print(pred)
                    preds[n] = "non-contradiction"
                    
                elif "self-contr" in pred:
                    preds[n] = "self-contradiction"

            #print(set(preds))
            pred_df["prediction"] = preds

            correct = []
            
            for pred, label in zip(preds, gold_labels):
                if pred == label:
                    correct.append(True)
                    
                else:
                    correct.append(False)

            pred_df["correct"] = correct

            acc = accuracy_score(gold_labels, preds)
            f1 = f1_score([1 if lab == "self-contradiction" else 0 for lab in gold_labels], [1 if lab == "self-contradiction" else 0 for lab in preds])
            conf_matrix = confusion_matrix(gold_labels, preds)
            #print(acc)
            #print(f1)

            if num == 2:
                accs[0].append(acc)
                f1s[0].append(f1)
            elif num == 4:
                accs[1].append(acc)
                f1s[1].append(f1)
            elif num == 8:
                accs[2].append(acc)
                f1s[2].append(f1)
            elif num == 16:
                accs[3].append(acc)
                f1s[3].append(f1)
            elif num == 32:
                accs[4].append(acc)
                f1s[4].append(f1)

            
            #with open(f"evaluations/{ty}_evaluation_{num}_{seed}_{run}", "w") as fout:
            #    fout.write(f"---Evaluation of run {run} of {ty} trial with {num} primes and random seed {seed} \n\n")
            #    fout.write(f"- self-contradiction primes: {prime_labels.count('self-contradiction')} \n")
            #    fout.write(f"- non-contradiction: {prime_labels.count('non-contradiction')}\n\n")

            #    fout.write(f"-accuracy: {acc} \n")
            #    fout.write(f"-F1-Score: {f1} \n\n")

                #fout.write("---Confusion matrix--- \n")
                #fout.write("\t \t pred pos \t pred neg \n")
                #fout.write(f"act pos \t {conf_matrix[0][0]} \t\t {conf_matrix[0][1]} \n")
                #fout.write(f"act neg \t {conf_matrix[1][0]} \t\t {conf_matrix[1][1]}")
            
            #with open(f"evaluations/{ty}_evaluation_all", "a") as fout:
             #   fout.write(f"+++Evaluation of run {run} of {ty} trial with {num} primes and random seed {seed}+++ \n\n")
             #   fout.write(f"- self-contradiction primes: {prime_labels.count('self-contradiction')} \n")
             #   fout.write(f"- non-contradiction: {prime_labels.count('non-contradiction')}\n\n")

              #  fout.write(f"-accuracy: {acc} \n")
              #  fout.write(f"-F1-Score: {f1} \n\n")

              #  fout.write("---Confusion matrix--- \n")
              #  fout.write("\t \t pred pos \t pred neg \n")
              #  fout.write(f"act pos \t {conf_matrix[0][0]} \t\t {conf_matrix[0][1]} \n")
               # fout.write(f"act neg \t {conf_matrix[1][0]} \t\t {conf_matrix[1][1]} \n\n")


trials = ["2-shot", "4-shot", "8-shot", "16-shot", "32-shot"]

#print(accs)
#print(f1s)

#with open(f"mean_std_all.tsv", "w") as fout:
#    writer = csv.writer(fout)
#    writer.writerow(["run", "f1", "std", "acc", "std"])
#    for i in range(len(trials)):
#        trial = trials[i]
#        f1_mean = mean(f1s[i])
#        f1_std = np.std(np.asanyarray(f1s[i]))
#        acc_mean = mean(accs[i])
#        acc_std = np.std(np.asarray(accs[i]))
#        writer.writerow([trial, f1_mean, f1_std, acc_mean, acc_std])

print("f1_2:", round(100*mean(f1s[0]),1))
print("f1_2_std:", round(100*np.std(np.asarray(f1s[0])),2))
print("acc_2:", round(100*mean(accs[0]),1))
print("acc_2_std:", round(100*np.std(np.asarray(accs[0]))),2)
print("f1_4:", round(100*mean(f1s[1]),1))
print("f1_4_std:", round(100*np.std(np.asarray(f1s[1])),2))
print("acc_4:", round(100*mean(accs[1]),1))
print("acc_4_std:", round(100*np.std(np.asarray(accs[1])),2))
print("f1_8:", round(100*mean(f1s[2]),1))
print("f1_8_std:", round(100*np.std(np.asarray(f1s[2])),2))
print("acc_8:", round(100*mean(accs[2]),1))
print("acc_8_std:", round(100*np.std(np.asarray(accs[2])),2))
print("f1_16:", round(100*mean(f1s[3]),1))
print("f1_16_std:", round(100*np.std(np.asarray(f1s[3])),2))
print("acc_16:", round(100*mean(accs[3]),1))
print("acc_16_std:", round(100*np.std(np.asarray(accs[3])),2))
print("f1_32:", round(100*mean(f1s[4]),1))
print("f1_32_std:", round(100*np.std(np.asarray(f1s[4])),2))
print("acc_32:", round(100*mean(accs[4]),1))
print("acc_32_std:", round(100*np.std(np.asarray(accs[4])),2))

f1_2: 56.7
f1_2_std: 3.67
acc_2: 54.1
acc_2_std: 1 2
f1_4: 58.3
f1_4_std: 4.64
acc_4: 54.1
acc_4_std: 1.0
f1_8: 57.9
f1_8_std: 7.0
acc_8: 54.3
acc_8_std: 0.77
f1_16: 59.0
f1_16_std: 3.83
acc_16: 53.8
acc_16_std: 0.57
f1_32: 60.1
f1_32_std: 5.19
acc_32: 53.3
acc_32_std: 0.83


In [14]:
for acc_l, f1_l in zip(accs, f1s):
    for acc, f1 in zip(acc_l, f1_l):
        print(f".{round(100*acc)} & .{round(100*f1)}")

.54 & .52
.54 & .52
.54 & .53
.55 & .59
.55 & .60
.55 & .60
.54 & .60
.55 & .61
.56 & .62
.52 & .54
.52 & .53
.54 & .55
.53 & .55
.53 & .55
.53 & .55
.56 & .53
.56 & .52
.55 & .52
.54 & .63
.54 & .64
.54 & .64
.54 & .62
.54 & .62
.53 & .61
.55 & .62
.55 & .62
.54 & .61
.55 & .46
.55 & .45
.56 & .47
.55 & .65
.53 & .64
.53 & .64
.54 & .59
.55 & .59
.54 & .60
.54 & .66
.54 & .66
.54 & .65
.54 & .56
.55 & .57
.54 & .56
.54 & .57
.53 & .56
.54 & .57
.54 & .58
.53 & .58
.53 & .57
.53 & .58
.54 & .59
.54 & .59
.54 & .53
.54 & .52
.54 & .52
.52 & .62
.53 & .63
.53 & .63
.53 & .67
.52 & .66
.53 & .67
